## StyleBlend Training

This script is used for trianing to capture the composition and texture styles from reference images.

### Prepare

In [ ]:
import os
from omegaconf import OmegaConf
from accelerate.utils import set_seed
from src.composition_style_training import create_plausible_dataset
from src.composition_style_training import train as train_cs
from src.texture_style_training import train as train_fs

config_path = './configs/training_config_sd21.yaml'
cfg = OmegaConf.load(config_path)
set_seed(cfg.seed)

# TODO: specify the style name
style = 'style1'

cfg.output_dir = os.path.join(cfg.output_dir, style)
os.makedirs(cfg.output_dir, exist_ok=True)

### Composition Style Learning

It first creates a plausible dataest using SDEdit, and then trains the LoRA-Text-Encoder on the generated data by a DreamBooth-like method.

In [ ]:
config_to_save = OmegaConf.create({
    'seed': cfg.seed,
    'output_dir': cfg.output_dir,
    'composition_style_training': cfg.composition_style_training
})
OmegaConf.save(config_to_save, os.path.join(cfg.output_dir, 'composition_style_training_config.yaml'))
print(f'Configurations saved to {os.path.join(cfg.output_dir, "composition_style_training_config.yaml")}')

if not os.path.exists(os.path.join(cfg.composition_style_training.dir_to_save_dataset, style)):
    create_plausible_dataset(cfg, style=style)
    
train_cs(cfg, style=style)

### Texture Style Learning

It first optimizes the style embeddings using Textual Inversion, and then trains the LoRA-Unet on the style data by DreamBooth-LoRA.

In [ ]:
config_to_save = OmegaConf.create({
    'seed': cfg.seed,
    'output_dir': cfg.output_dir,
    'texture_style_training': cfg.texture_style_training
})
OmegaConf.save(config_to_save, os.path.join(cfg.output_dir, 'texture_style_training_config.yaml'))
print(f'Configurations saved to {os.path.join(cfg.output_dir, "texture_style_training_config.yaml")}')

train_fs(cfg, style=style)